In [28]:
import os
import zipfile
import random
import tensorflow as tf
import shutil
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
from os import getcwd

#!pip install -q -U keras-tuner
import keras_tuner as kt


In [29]:
print(getcwd())
path_arab_words = f"{getcwd()}/../AHDB.zip"
shutil.rmtree('/tmp')

local_zip = path_arab_words
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

/content


In [30]:
print(len(os.listdir('/tmp/AHDB/AAN/')))

34


In [31]:
try:
   
    base_dir = "/tmp/AHDB"

    train_dir = os.path.join(base_dir, 'training')
    validation_dir = os.path.join(base_dir, 'validation')
    testing_dir = os.path.join(base_dir, 'testing')
    
    s=['AAN','ABD','ALA','ALADHI','ALAM','ALLAH','ALLATI','ALYAWM','AN','AW','FI'
    ,'HADHA','HADHIHI','HIA','HOUNAKA','HOWA','ILA','KABLA','KAD','KAMA'
    ,'KANA','KHILALA','MA','MAA','MAN','MOHAMED','TOMA','YAKON']


    # a loop that creates a validation and a training dir for each classe
    # ex train_aan_dir = os.path.join(train_dir, 'AAN')
    for k in s:

      exec(f'train_{k}_dir = os.path.join(train_dir, k)')
      exec(f'validation_{k}_dir = os.path.join(validation_dir, k)')
      exec(f'test_{k}_dir = os.path.join(testing_dir, k)')
      valid = '/tmp/recognize/validation/'+k+'/'
      train ='/tmp/recognize/training/'+k+'/'
      test ='/tmp/recognize/testing/'+k+'/'
      os.makedirs(test)
      os.makedirs(train)
      os.makedirs(valid)
     


except OSError:
    pass

In [32]:
def split_data(SOURCE, TRAINING,VALIDATION, TESTING, TRAIN_SIZE, VAL_SIZE):

    listdir= os.listdir (SOURCE)
    list_source= random.sample(listdir,len(listdir))
    for i in range (len (list_source)):
        file_source = os.path.join(SOURCE,list_source[i-1])
        file_trainin = os.path.join(TRAINING,list_source[i-1])
        file_valid = os.path.join(VALIDATION,list_source[i-1])
        file_tastin = os.path.join(TESTING,list_source[i-1])
        if (i<len (list_source)*TRAIN_SIZE):
            if (os.path.getsize(file_source))!=0:
                #copy it
                copyfile(file_source,file_trainin)
        elif i>= len (list_source)*TRAIN_SIZE and i <len (list_source)*VAL_SIZE :
             if (os.path.getsize(file_source))!=0:
                #copy it
                copyfile(file_source,file_valid)
        else :
          if (os.path.getsize(file_source))!=0:
                #copy it
                copyfile(file_source,file_tastin)


train_size = .60  # 90% training and the others for test 
valid_size = .80  
    
    
for k in s:
      source_dir="/tmp/AHDB/"+k+"/"
      testing_dir = '/tmp/recognize/testing/'+k+'/'
      train_dir ='/tmp/recognize/training/'+k+'/'
      validation_dir ='/tmp/recognize/validation/'+k+'/'
      split_data(source_dir, train_dir,validation_dir, testing_dir, train_size,valid_size)


In [33]:
# DEFINE A KERAS MODEL TO CLASSIFY THE 28 CLASSES
# USING 3 CONVOLUTION LAYERS


#_______________tweeking parameters____________
# Tune the number of units in the first Dense layer
# Choose an optimal value between 32-512

def model_builder(hp):
    
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  hp_activation=hp.Choice('activation',values=["relu","elu"])
  model = tf.keras.models.Sequential([

      # Note the input shape is the desired size of the image 150x150 with 3 bytes color
      tf.keras.layers.Conv2D(16, (3,3), activation=hp_activation, input_shape=(150, 150, 3)),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Conv2D(32, (3,3), activation=hp_activation),
      tf.keras.layers.MaxPooling2D(2,2), 
      tf.keras.layers.Conv2D(64, (3,3), activation=hp_activation), 
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Conv2D(128, (3,3), activation=hp_activation), 
      tf.keras.layers.MaxPooling2D(2,2),
      # Flatten the results to feed into a DNN
      tf.keras.layers.Flatten(), 
      #  hidden layer
      tf.keras.layers.Dense(hp_units, activation=hp_activation), 
      # Only 1 output neuron. It will contain a value from 0 or 1 where 1 in the indice of the predicted class
      tf.keras.layers.Dense(28, activation='softmax') 
  ])

  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
  model.compile(optimizer=RMSprop(lr=hp_learning_rate), loss='categorical_crossentropy', metrics=['acc'])

  return model



 

In [34]:
TRAINING_DIR = '/tmp/recognize/training/'
train_datagen = ImageDataGenerator( rescale = 1.0/255. )
train_generator =train_datagen.flow_from_directory(TRAINING_DIR,
                                                   batch_size=10,
                                                    class_mode='categorical',
                                                    target_size=(150, 150))  


VALIDATION_DIR ='/tmp/recognize/validation/' 
validation_datagen = ImageDataGenerator( rescale = 1.0/255. )
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR,
                                                         batch_size=10,
                                                         class_mode  = 'categorical',
                                                         target_size = (150, 150))



TESTING_DIR ='/tmp/recognize/testing/' 
testing_datagen = ImageDataGenerator( rescale = 1.0/255. )
testing_generator = validation_datagen.flow_from_directory(TESTING_DIR,
                                                         batch_size=10,
                                                         class_mode  = 'categorical',
                                                         target_size = (150, 150))
                       


''' training_datagen = ImageDataGenerator(
      rescale = 1./255,
	    rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      #horizontal_flip=True,
      brightness_range=[0.4,1.5],
      fill_mode='nearest')
validation_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = training_datagen.flow_from_directory(
	TRAINING_DIR,
	target_size=(150,150),
	class_mode='categorical',
  batch_size=10
)

validation_generator = validation_datagen.flow_from_directory(
	VALIDATION_DIR,
	target_size=(150,150),
	class_mode='categorical',
  batch_size=10
) '''

# Expected Output:
# Found 2700 images belonging to 2 classes.
# Found 300 images belonging to 2 classes.


Found 581 images belonging to 28 classes.
Found 195 images belonging to 28 classes.
Found 180 images belonging to 28 classes.


" training_datagen = ImageDataGenerator(\n      rescale = 1./255,\n\t    rotation_range=40,\n      width_shift_range=0.2,\n      height_shift_range=0.2,\n      shear_range=0.2,\n      zoom_range=0.2,\n      #horizontal_flip=True,\n      brightness_range=[0.4,1.5],\n      fill_mode='nearest')\nvalidation_datagen = ImageDataGenerator(rescale = 1./255)\n\ntrain_generator = training_datagen.flow_from_directory(\n\tTRAINING_DIR,\n\ttarget_size=(150,150),\n\tclass_mode='categorical',\n  batch_size=10\n)\n\nvalidation_generator = validation_datagen.flow_from_directory(\n\tVALIDATION_DIR,\n\ttarget_size=(150,150),\n\tclass_mode='categorical',\n  batch_size=10\n) "

In [35]:
tuner = kt.Hyperband(model_builder,
                     objective='val_acc',
                     max_epochs=15,
                     factor=3,
                      overwrite=True
                     #directory='my_dir',
                     #project_name='intro_to_kt'
                     )

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

tuner.search(train_generator, epochs=50,validation_data=validation_generator, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")
''' 
history = model.fit_generator(train_generator,
                              epochs=10,
                              verbose=1,
                              validation_data=validation_generator) '''
# 1h 20 min

Trial 30 Complete [00h 02m 44s]
val_acc: 0.03589743748307228

Best val_acc So Far: 0.764102578163147
Total elapsed time: 00h 43m 16s
INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 224 and the optimal learning rate for the optimizer
is 0.001.



' \nhistory = model.fit_generator(train_generator,\n                              epochs=10,\n                              verbose=1,\n                              validation_data=validation_generator) '

In [36]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(train_generator, epochs=50,validation_data=validation_generator)

val_acc_per_epoch = history.history['val_acc']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

model.summary()

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/50
59/59 [==============================] - 17s 278ms/step - loss: 3.3936 - acc: 0.0585 - val_loss: 3.1367 - val_acc: 0.1897
Epoch 2/50
59/59 [==============================] - 16s 269ms/step - loss: 2.1871 - acc: 0.3873 - val_loss: 1.3692 - val_acc: 0.6154
Epoch 3/50
59/59 [==============================] - 16s 272ms/step - loss: 0.8017 - acc: 0.7780 - val_loss: 1.2522 - val_acc: 0.6308
Epoch 4/50
59/59 [==============================] - 16s 271ms/step - loss: 0.2968 - acc: 0.9105 - val_loss: 1.0499 - val_acc: 0.6923
Epoch 5/50
59/59 [==============================] - 16s 269ms/step - loss: 0.1075 - acc: 0.9725 - val_loss: 1.3697 - val_acc: 0.7385
Epoch 6/50
59/59 [==============================] - 16s 270ms/step - loss: 0.0704 - acc: 0.9776 - val_loss: 1.1104 - val_acc: 0.7641
Epoch 7/50
59/59 [==============================] - 16s 267ms/step - loss: 0.0576 - acc: 0.9845 - val_loss: 1.4797 - val_acc: 0.7436
Epoch 8/50
59/59 [==============================] - 16s 272ms/step - 

KeyboardInterrupt: ignored

In [ ]:
print(best_hps.get('activation'))
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hyperhistory=hypermodel.fit(train_generator, epochs=best_epoch,validation_data=validation_generator)

In [ ]:
# PLOT LOSS AND ACCURACY
%matplotlib inline

import matplotlib.image  as mpimg
import matplotlib.pyplot as plt

#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
acc=hyperhistory.history['acc']
val_acc=hyperhistory.history['val_acc']
loss=hyperhistory.history['loss']
val_loss=hyperhistory.history['val_loss']

epochs=range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot(epochs, acc, 'r', "Training Accuracy")
plt.plot(epochs, val_acc, 'b', "Validation Accuracy")
plt.title('Training and validation accuracy')
plt.figure()

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot(epochs, loss, 'r', "Training Loss")
plt.plot(epochs, val_loss, 'b', "Validation Loss")


plt.title('Training and validation loss')


# Desired output. Charts with training and validation metrics. No crash :)

In [ ]:
#evaluate the model with test data

#get the labels
x_test,y_test=testing_generator.next()
eval_result = hypermodel.evaluate(x_test, y_test)
print("[test loss, test accuracy]:", eval_result)



In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image



!mkdir -p "/content/saved_model"
hypermodel.save('/content/saved_model/my_model_2') 
!zip -r '/content/saved_model/file_no_2.zip' '/content/saved_model/my_model_2'
from google.colab import files
#files.download("/content/saved_model/file_no.zip")


uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(150, 150))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = hypermodel.predict(images, batch_size=10)
  print("---------------Found classes ----------------")
  print( classes[0])
  print( classes[0][0])
  for i in range (len(classes[0])):
    if (1-classes[0][i])<0.1:
      print(s[i])